In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import os

home_dir = os.path.expanduser('~/InsureAI')
# Define the path to save the fine-tuned model
model_dir = os.path.join(home_dir, "models", "fine_tuned_model")  # Points to InsureAI/models/fine_tuned_model

/home/waijianlim/miniconda3/envs/DeepSeek/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from unsloth import FastLanguageModel

# model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit"
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    # device_map="auto",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu118. CUDA: 7.5. CUDA Toolkit: 11.8. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Prepare Company Data Informations for fine-tuning

In [3]:
# Initialize insurance.db data
import subprocess

# Define the path to setup.py relative to the notebook
setup_script_path = os.path.abspath("../sql/setup.py")

# Run the setup.py script
try:
    subprocess.run(["python", setup_script_path], check=True)
    print("setup.py executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"An error occurred while executing setup.py: {e}")


Products already exist. No new records inserted.
(1, 'Death A', 'Death', '{"coverage": "term", "premium": 100, "SA":10000}', 'InsureAI')
(2, 'Death B', 'Death', '{"coverage": "endowment", "premium": 200, "SA":20000}', 'InsureAI')
(3, 'Death C', 'Death', '{"coverage": "full", "premium": 300, "SA":30000}', 'InsureAI')
(4, 'TPD A', 'TPD', '{"coverage": "term", "premium": 100, "SA":10000}', 'InsureAI')
(5, 'TPD B', 'TPD', '{"coverage": "endowment", "premium": 200, "SA":20000}', 'InsureAI')
(6, 'TPD C', 'TPD', '{"coverage": "full", "premium": 300, "SA":30000}', 'InsureAI')
(7, 'Critical Illness A', 'Critical Illness', '{"coverage": "term", "premium": 100, "SA":10000}', 'InsureAI')
(8, 'Critical Illness B', 'Critical Illness', '{"coverage": "endowment", "premium": 200, "SA":20000}', 'InsureAI')
(9, 'Critical Illness C', 'Critical Illness', '{"coverage": "full", "premium": 300, "SA":30000}', 'InsureAI')
(10, 'Accidental A', 'Accidental', '{"coverage": "term", "premium": 100, "SA":10000}', 'In

In [4]:
import json
import sqlite3
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split

# Configuration
NUM_VARIATIONS_PER_ITEM = 313  # 219 + 94 = 313 (70% + 30%)
TRAIN_TEST_SPLIT = 0.3
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Initialize database connection (replace with your actual path)
home_dir = os.path.expanduser("~/InsureAI")
conn = sqlite3.connect(os.path.join(home_dir, 'insurance.db'))
cursor = conn.cursor()

# Fetch products from database
cursor.execute("SELECT * FROM products")
products = cursor.fetchall()

# Base company data
company_data = {
    "company_name": "InsureAI",
    "launch_date": "1995-05-15",
    "description": "InsureAI is a leading provider of life, health, and general insurance products in Singapore. We are committed to helping our customers achieve financial security and peace of mind.",
    "company_type": "Life and General Insurance",
    "headquarters": "Singapore",
    "website": "https://www.insureai.sg",
    "contact_email": "info@insureai.sg",
    "contact_phone": "+65 8888 1314",
    "products": []
}

# Process products
for product in products:
    product_id, name, types, features, company = product
    features_dict = json.loads(features)
    
    product_details = {
        "id": product_id,
        "name": name,
        "types": types,
        "features": features_dict,
        "base_description": f"{name} is a type of {types} insurance. It is a/an {features_dict.get('coverage')} insurance product offered by {company}. Key features include: {', '.join([f'{k}: {v}' for k, v in features_dict.items()])}."
    }
    company_data["products"].append(product_details)

# Variation generation functions
def generate_company_variations(company, num_variations):
    variations = []
    
    # Prompt templates
    prompt_templates = [
        "Tell me about {name}",
        "Describe {name}",
        "What can you tell me about {name}?",
        "Provide information about {name}",
        "Give me an overview of {name}",
        "What is {name} known for?",
        "Share details about {name}",
        "Can you explain what {name} does?",
        "I need information about {name}",
        "What makes {name} unique?",
        "How would you describe {name}?",
        "What should I know about {name}?",
        "Please introduce {name}",
        "Explain the company {name}",
        "What are the key facts about {name}?"
    ]
    
    # Completion components
    components = {
        'type': [
            f"{company['company_name']} is a {company['company_type']} company",
            f"A {company['company_type']} provider: {company['company_name']}",
            f"Operating as a {company['company_type']} company"
        ],
        'hq': [
            f"headquartered in {company['headquarters']}",
            f"based in {company['headquarters']}",
            f"with its main office in {company['headquarters']}"
        ],
        'launch': [
            f"Established in {company['launch_date']}",
            f"Founded on {company['launch_date']}",
            f"Operating since {company['launch_date']}"
        ],
        'desc': [
            company['description'],
            company['description'].replace("leading", "prominent"),
            company['description'].replace("helping", "assisting")
        ],
        'contact': [
            f"Contact: {company['contact_email']} | {company['contact_phone']} | {company['website']}",
            f"Reach them at {company['contact_phone']} or {company['contact_email']}",
            f"Website: {company['website']} | Email: {company['contact_email']}"
        ]
    }

    for _ in range(num_variations):
        # Generate prompt
        prompt = random.choice(prompt_templates).format(name=company['company_name'])
        
        # Generate completion with shuffled components
        completion_parts = [
            random.choice(components['type']),
            random.choice(components['hq']),
            random.choice(components['launch']),
            random.choice(components['desc']),
            random.choice(components['contact'])
        ]
        random.shuffle(completion_parts)
        completion = ". ".join(completion_parts) + "."
        
        variations.append((prompt, completion))
    
    return variations

def generate_product_variations(product, company_name, num_variations):
    variations = []
    
    # Prompt templates
    prompt_templates = [
        "Describe {product} from {company}",
        "What does {product} by {company} offer?",
        "Explain the {product} insurance policy",
        "What are the features of {product}?",
        "Tell me about {company}'s {product}",
        "What coverage does {product} provide?",
        "Details about the {product} plan",
        "What makes {product} by {company} special?",
        "Information about {product} insurance",
        "Why choose {product} from {company}?"
    ]
    
    # Completion components
    components = {
        'type': [
            f"{product['name']} is a {product['types']} insurance product",
            f"This {product['types']} policy: {product['name']}",
            f"A {product['types']} solution: {product['name']}"
        ],
        'coverage': [
            f"Coverage type: {product['features']['coverage']}",
            f"{product['features']['coverage'].title()} coverage",
            f"Provides {product['features']['coverage']} protection"
        ],
        'premium': [
            f"Premium: ${product['features']['premium']}",
            f"Cost: {product['features']['premium']} SGD",
            f"Priced at {product['features']['premium']} SGD"
        ],
        'sa': [
            f"Sum assured: {product['features']['SA']} SGD",
            f"Coverage amount: {product['features']['SA']}",
            f"SA: {product['features']['SA']}"
        ],
        'desc': [
            f"Offered by {company_name}",
            "Designed for comprehensive protection",
            "Includes valuable benefits",
            "Flexible policy options"
        ]
    }

    for _ in range(num_variations):
        # Generate prompt
        prompt = random.choice(prompt_templates).format(
            product=product['name'], 
            company=company_name
        )
        
        # Generate completion
        completion_parts = [
            random.choice(components['type']),
            random.choice(components['coverage']),
            random.choice(components['premium']),
            random.choice(components['sa']),
            random.choice(components['desc'])
        ]
        random.shuffle(completion_parts)
        completion = ". ".join(completion_parts) + "."
        
        variations.append((prompt, completion))
    
    return variations

# Generate and split data
all_train = []
all_eval = []

# Company variations
company_variations = generate_company_variations(company_data, NUM_VARIATIONS_PER_ITEM)
train_company, eval_company = train_test_split(
    company_variations, 
    test_size=TRAIN_TEST_SPLIT,
    random_state=SEED
)
all_train.extend(train_company)
all_eval.extend(eval_company)

# Product variations
for product in company_data["products"]:
    product_variations = generate_product_variations(
        product,
        company_data["company_name"],
        NUM_VARIATIONS_PER_ITEM
    )
    train_product, eval_product = train_test_split(
        product_variations,
        test_size=TRAIN_TEST_SPLIT,
        random_state=SEED
    )
    all_train.extend(train_product)
    all_eval.extend(eval_product)

# Shuffle datasets
random.shuffle(all_train)
random.shuffle(all_eval)

# Save to JSONL files
def save_jsonl(data, filename):
    with open(filename, "w") as f:
        for prompt, completion in data:
            f.write(json.dumps({
                "prompt": "[INSURANCE_QUERY] " + prompt + "\n\n###\n\n",
                "completion": " " + completion + " END"
            }) + "\n")

save_jsonl(all_train, "train_data.jsonl")
save_jsonl(all_eval, "eval_data.jsonl")

print(f"Training samples: {len(all_train)}")
print(f"Evaluation samples: {len(all_eval)}")
print("Files saved successfully!")

Training samples: 3504
Evaluation samples: 1504
Files saved successfully!


In [5]:
# Load the fine-tuning data
# dataset = load_dataset("json", data_files="fine_tuning_data.jsonl")
dataset = load_dataset("json", data_files={"train": "train_data.jsonl", "test": "eval_data.jsonl"})

Generating train split: 3504 examples [00:00, 604783.39 examples/s]
Generating test split: 1504 examples [00:00, 387364.64 examples/s]


In [6]:
# Tokenize the dataset
def preprocess_function(examples):
    inputs = tokenizer(examples["prompt"], max_length=max_seq_length, truncation=True, padding="max_length")
    outputs = tokenizer(examples["completion"], max_length=max_seq_length, truncation=True, padding="max_length")
    inputs["labels"] = outputs["input_ids"]  # Set the response as labels for training
    return inputs
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|█████████████████████████████████████████████████████████████████| 1504/1504 [00:01<00:00, 1033.71 examples/s]


In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "v_proj"],  # Fine-tune key attention layers
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.1.8 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [8]:
training_args = TrainingArguments(
    output_dir="./results",  # Where to save model checkpoints
    overwrite_output_dir=True,  # Overwrite old checkpoints
    per_device_train_batch_size=1,  # Small batch to fit in 6GB VRAM
    per_device_eval_batch_size=1,  # Same for evaluation
    gradient_accumulation_steps=8,  # Helps with small batch size
    eval_strategy="steps",  # Evaluate every few steps
    eval_steps=500,  # Frequency of evaluation
    save_steps=500,  # Save model every 500 steps
    save_total_limit=2,  # Keep only last 2 checkpoints
    logging_dir="./logs",  # Log directory
    logging_steps=100,  # Log every 100 steps
    learning_rate=2e-5,  # Suitable for Qwen fine-tuning
    num_train_epochs=5,  # Number of epochs
    weight_decay=0.01,  # Regularization
    fp16=True,  # Use mixed precision to save VRAM
    optim="adamw_bnb_8bit",  # 8-bit optimizer for efficiency
    lr_scheduler_type="cosine",  # Learning rate decay
    warmup_steps=100,  # Gradual increase in learning rate
    report_to="none",  # Avoid logging to external tools
)


In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipykernel_3836/3298343692.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,504 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 8,760
 "-____-"     Number of trainable parameters = 2,179,072


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluate the model
import math

eval_results = trainer.evaluate()
print(eval_results)

perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity}")

In [ ]:
# Save the model and tokenizer
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

print(f"Fine-tuning complete. Model saved to {model_dir}.")